In [1]:
import sqlite3
import pandas as pd

In [2]:
df = pd.read_excel('RIVER_LAKE_DATABASE4.xlsx')
df.head(5)

,LAND,KANTON,GEWAESSERNAME,KLASSE,TYP,COORDINATES,Unnamed: 6,OWNER,REVIER NR,SAISON,Kontakt Name,Kontakt Strasse,Kontakt PLZ,Kontakt Ort,Unnamed: 14,Unnamed: 15,Klassen,Unnamed: 17,Typ,Unnamed: 19
0,Schweiz,SO,Dünnern,1,0.0,"(47.34912926867982, 7.904297420108378),(47.349...",=,NaN,NaN,16.03-01.10,NaN,NaN,NaN,NaN,NaN,NaN,0,Fluss,0.0,Frei
1,Schweiz,SO,Dünnern Schongebiet,3,0.0,"(47.34905733154134, 7.903928547929127),(47.349...",=,NaN,NaN,not appl.,NaN,NaN,NaN,NaN,NaN,NaN,1,Bach,1.0,Privat
2,Schweiz,SO,Aare A1,0,0.0,"(47.190572317610155, 7.475212656614758),(47.19...",=,NaN,NaN,ganzes Jahr,NaN,NaN,NaN,NaN,NaN,NaN,2,Bächlein,2.0,Gemischt (Freiangelei)
3,Schweiz,SO,Aare A1-2,0,0.0,"(47.225410993236935, 7.57915437966949),(47.224...",=,NaN,NaN,ganzes Jahr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Schweiz,SO,Aare A2,0,0.0,"(47.36561829849677, 7.921917925016032),(47.367...",=,NaN,NaN,ganzes Jahr,NaN,NaN,NaN,NaN,NaN,NaN,3,Schongebeit,NaN,NaN


In [3]:
list(set(df['SAISON']))

['16.03-01.10', nan, 'ganzes Jahr', '01.03-15.10', 'not appl.']

In [4]:
db_name = 'fishapp.db'  # Replace with your database name
connection = sqlite3.connect(db_name)
cursor = connection.cursor()

In [5]:
#df_sub = df[df['GEWAESSERNAME'] == 'Dünnern']
df_sub = df
unique_owners = []
wasser_id = 1

for j in df_sub.index:
    coords = df_sub.loc[j,'COORDINATES']
    name = df_sub.loc[j,'GEWAESSERNAME']
    klasse = int(df_sub.loc[j,'KLASSE'])
    land = 'CH'
    kanton = df.loc[j,'KANTON']
    saison = df.loc[j,'SAISON']
    owner_id = 1

    kontakt_name = df.loc[j,'Kontakt Name']
    kontakt_str = df.loc[j,'Kontakt Strasse']
    kontakt_plz = df.loc[j,'Kontakt PLZ']
    kontakt_city = df.loc[j,'Kontakt Ort']
    
    # logical data    
    if int(klasse) == 3:
        schoengebiet = True
    else:
        schongebiet = False
    if klasse == 0:
        water_type = 'Fluss'
    if klasse == 1 or klasse == 2:
        water_type = 'Bach'
    if saison ==  '01.03-15.10':
        saion_id = 3
    if saison == 'ganzes Jahr':
        saison_id = 2
    if saison == '16.03-01.10':
        saison_id = 1
    if str(saison) == 'nan' or saison == 'not appl.':
        saison_id = 0

    data_wasser = (wasser_id, land, kanton, 1, name, water_type, '', schongebiet, saison_id)
    print(data_wasser)

    # only continue if coordinates available
    if not str(coords) == 'nan':
        if not len(coords.strip()) == 0:

            # insert water
            cursor.execute('''
            INSERT INTO Water (water_id, water_country, water_region, water_owner_id, water_name, water_type, water_sub, schongebiet, water_season_id)
            VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?);
            ''', data_wasser)
            # VALUES(1, 'CH', 'SO', 1, 'Dünnern', 'Bach', 'D', FALSE);

            # insert owner
            if not str(kontakt_name) == 'nan':
                if not kontakt_name in unique_owners:
                    owner_query = '''
                    INSERT INTO Water_Owners
                    (Name, Address_Street, Address_PLZ, Address_City, Address_Country, Owner_ID)
                    VALUES(?, ?, ?, ?, ?, ?);
                    '''
                    owner_data = (kontakt_name, kontakt_str, kontakt_plz, kontakt_city, 'CH', owner_id)
                    cursor.execute(owner_query, owner_data)
                    unique_owners.append(kontakt_name)
                    owner_id = owner_id + 1
          
            
            # adding coordinates
            coord_tuples = coords.split('),')
            for en, coord_tuple in enumerate(coord_tuples):
                coords = (coord_tuple[1:])
                x_coord = coords.split(',')[0].strip()
                if en == len(coord_tuples):
                    y_coord = coords.split(',')[1].strip()[:-1]
                else:
                    y_coord = coords.split(',')[1].strip()
                #print(f'x={x_coord} y={y_coord}')
                sql = '''
                INSERT INTO Water_Coords
                (water_id, coord_x, coord_y)
                VALUES(?, ?, ?);
                '''
                data = (wasser_id, x_coord, y_coord)
                cursor.execute(sql, data)
                
            wasser_id = wasser_id + 1

(1, 'CH', 'SO', 1, 'Dünnern', 'Bach', '', False, 1)


OperationalError: 8 values for 9 columns

In [ ]:
connection.commit()


In [ ]:
connection.close()
